# Getting Started with Taipy Core on Notebooks

!!! important "Supported Python versions"

    Taipy requires **Python 3.8** or newer.

Welcome to the **Getting Started** guide for Taipy Core. This tour shows you how to use Taipy Core to orchestrate pipelines. Taipy Core implements a modern backend for any data-driven application based on your business case.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_00/imd_end_interface.png width=700>
</div>

# Taipy Core

Taipy Core is one of the components of Taipy to facilitate pipeline orchestration. There are a lot of reasons for using Taipy Core:

- Taipy Core efficiently manages the execution of your functions/pipelines.

- Taipy Core manages data sources and monitors KPIs.

- Taipy Core provides easy management of multiple pipelines and end-user scenarios, which comes in handy in the context of Machine Learning or Mathematical optimization.

To apprehend the Scenario Management aspect of Taipy, you need to understand four essential concepts.

Each step of the **"Getting Started"** will focus on basic concepts of *Taipy*. Note that every step is dependent on 
the code of the previous one. After completing the last step, you will have the skills to develop your own Taipy 
application. 

## Before we begin

Only Taipy has to be installed. **Taipy** package requires Python 3.8 or newer;



In [0]:
# !pip install taipy


## Using Notebooks


# Configuration and execution
## Four fundamental concepts in Taipy Core:
- Data Nodes: are the translation of variables in Taipy. Data Nodes don't contain the data but know how to retrieve it. They can refer to any data: any Python object (string, int, list, dict, model, dataframe, etc), a Pickle file, a CSV file, an SQL database, etc. They know how to read and write data. You can even write your own custom Data Node if needed to access a particular data format.

- Tasks: are the translation of functions in Taipy.

- Pipelines: are a list of tasks executed with intelligent scheduling created automatically by Taipy. They usually represent a sequence of Tasks/functions corresponding to different algorithms like a simple baseline Algorithm or a more sophisticated Machine-Learning pipeline.

- Scenarios: End-Users very often require modifying various parameters to reflect different business situations. Taipy Scenarios will provide the framework to "play"/"execute" pipelines under different conditions/variations (i.e., data/parameters modified by the end user)


## What is a configuration?

Configuration is the structure or model of what is our scenario. It represents our Direct Acyclic Graph but also how we want our data to be stored or how our code is run. Taipy is able to create multiple instances of this structure with different data thus, we need a way to define it through this configuration step.


Let's create our first configuration and then create our entities to submit through Taipy Studio or direct Python Code.



In [1]:
from taipy import Config
import taipy as tp

# Normal function used by Taipy
def double(nb):
    return nb * 2



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_01/config_01.svg width=700>
</div>

- Two Data Nodes are being configured ('input' and 'output'). The 'input' Data Node has a _default_data_ put at 21. They will be stored as Pickle files by default, and are unique to their scenario.

- The task links the two Data Nodes through the Python function _double_.

- The pipeline will contain this one task, and the scenario will contain this one pipeline.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_01/config_01.gif width=700>
</div>


=== "Python configuration"

Here is the code to configure a simple scenario.



In [2]:
# Configuration of Data Nodes
input_data_node_cfg = Config.configure_data_node("input", default_data=21)
output_data_node_cfg = Config.configure_data_node("output")

# Configuration of tasks
task_cfg = Config.configure_task("double",
                                 double,
                                 input_data_node_cfg,
                                 output_data_node_cfg)

# Configuration of the pipeline and scenario
pipeline_cfg = Config.configure_pipeline("my_pipeline", [task_cfg])
scenario_cfg = Config.configure_scenario("my_scenario", [pipeline_cfg])


In [3]:
# Run of the Core
tp.Core().run()

# Creation of the scenario and execution
scenario = tp.create_scenario(scenario_cfg)
tp.submit(scenario)

print("Value at the end of task", scenario.output.read())
